In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
pd.set_option('display.max_columns',None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing data. --  There is no Color col

## Download the processed data from my github

In [11]:

train_path ='https://raw.githubusercontent.com/Mahsarnzh/Don-t-Get-Kicked/Ziang/Dec-7-result-withoutColor/Train_file_afterEncoding.csv'
train_df = pd.read_csv(train_path, index_col=False, error_bad_lines=False)

targets_path = 'https://raw.githubusercontent.com/Mahsarnzh/Don-t-Get-Kicked/Ziang/Dec-7-result-withoutColor/targets.csv'
targets = pd.read_csv(targets_path, index_col=False, error_bad_lines=False)

test_path = 'https://raw.githubusercontent.com/Mahsarnzh/Don-t-Get-Kicked/Ziang/Dec-7-result-withoutColor/Test_file_afterEncoding.csv'
test_df = pd.read_csv(test_path, index_col=False, error_bad_lines=False)

submissions_df_path = 'https://raw.githubusercontent.com/Mahsarnzh/Don-t-Get-Kicked/Ziang/me2300_data/sample_submission.csv'
submissions_df = pd.read_csv(submissions_df_path, index_col=False, error_bad_lines=False)

train_df.tail(3)



,VehicleAge,WheelTypeID,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,IsOnlineSale,Make_ACURA,Make_BUICK,Make_CADILLAC,Make_CHEVROLET,Make_CHRYSLER,Make_DODGE,Make_FORD,Make_GMC,Make_HONDA,Make_HYUNDAI,Make_INFINITI,Make_ISUZU,Make_JEEP,Make_KIA,Make_LEXUS,Make_LINCOLN,Make_MAZDA,Make_MERCURY,Make_MINI,Make_MITSUBISHI,Make_NISSAN,Make_OLDSMOBILE,Make_PONTIAC,Make_SATURN,Make_SCION,Make_SUBARU,Make_SUZUKI,Make_TOYOTA,Make_TOYOTA SCION,Make_VOLKSWAGEN,Make_VOLVO,Model_high1,Model_high2,Model_low,Model_mid1,Model_mid2,Model_mid3,Trim_high1,Trim_high2,Trim_low,Trim_mid1,Trim_mid2,Trim_mid3,SubModel_high1,SubModel_high2,SubModel_low,SubModel_mid1,SubModel_mid2,SubModel_mid3,Transmission_AUTO,Transmission_MANUAL,Transmission_nan,Nationality_AMERICAN,Nationality_OTHER,Nationality_TOP LINE ASIAN,Size_high1,Size_high2,Size_low,Size_mid1,Size_mid2,Size_mid3,VehBCost_high1,VehBCost_high2,VehBCost_low,VehBCost_mid1,VehBCost_mid2,VehBCost_mid3,WarrantyCost_high1,WarrantyCost_high2,WarrantyCost_low,WarrantyCost_mid1,WarrantyCost_mid2,WarrantyCost_mid3
25307,2,22.0,8132.0,9389.0,12889.0,14036.0,8344.0,9618.0,12807.0,13642.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25308,6,22.0,3935.0,5111.0,4750.0,6020.0,3596.0,4893.0,4384.0,5784.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
25309,3,22.0,10257.0,11660.0,13867.0,14963.0,10290.0,11680.0,13921.0,15213.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Spliting Train and targets for local evaluation

In [12]:
test_df_numeric_cols = test_df.select_dtypes(exclude='object').columns.tolist()
train_df_numeric_cols = train_df.select_dtypes(exclude='object').columns.tolist()

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# imputer=SimpleImputer(strategy='mean');
imputer = IterativeImputer(random_state=4)

imputer.fit(train_df[train_df_numeric_cols]);
train_df[train_df_numeric_cols]=imputer.transform(train_df[train_df_numeric_cols]);

imputer.fit(train_df[test_df_numeric_cols]);
test_df[test_df_numeric_cols]=imputer.transform(test_df[test_df_numeric_cols]);

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning:

[IterativeImputer] Early stopping criterion not reached.



# MODEL TESTING!!!!

## toy-try - RandomForestClassifier

In [13]:
# train_df.to_csv('Train_df.csv',index=False)

train_df_backUp = train_df
targets_backUp = targets
test_df_backUp = test_df
from sklearn.ensemble import RandomForestClassifier    
model = RandomForestClassifier()
model.fit(train_df_backUp,targets_backUp)

preds = model.predict_proba(test_df_backUp)
submissions_df['Predicted'] = preds[:,1]

submissions_df.to_csv('Submissions_RDF_Dec6-3.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [14]:
preds

array([[0.45 , 0.55 ],
       [0.81 , 0.19 ],
       [0.254, 0.746],
       ...,
       [0.81 , 0.19 ],
       [0.98 , 0.02 ],
       [0.91 , 0.09 ]])

## Auto-downloader [Chrome!]

In [ ]:
## When you run it in coLab and use google chrome as the browser. 
from google.colab import files
files.download('Submissions_SVC_Dec6.csv')

In [ ]:
# SVC